<h1>Test set path</h1>

In [ ]:
import sys
sys.path.append("C:/Users/Theo/Documents/Unif/ChimpRec/Code")


from chimplib.metric import *
from chimplib.imports import os, cv2, plt

test_set = "..." #insert path
model_path = "..." #insert path

<h3>Generic function to compute intersection over union.</h3>

In [5]:
print(iou([0.386891, 0.573836, 0.274225, 0.328799], [0.324225, 0.497304, 0.400000, 0.482353])) # BIG
print(iou([0.670643, 0.656066, 0.136706, 0.164583], [0.639254, 0.618382, 0.199631, 0.240196])) # MEDIUM
print(iou([0.820532, 0.697243, 0.068833, 0.082230], [0.805022, 0.678125, 0.100000, 0.120956])) # SMALL
print(iou([0.902880, 0.717892, 0.033973, 0.040931], [0.895362, 0.709050, 0.049453, 0.058860])) # TINY

0.46731805220968897
0.46922246572820714
0.4679501297992655
0.4777199233072285


In [6]:
print(weighted_iou([0.386891, 0.573836, 0.274225, 0.328799], [0.324225, 0.497304, 0.400000, 0.482353])) # BIG
print(weighted_iou([0.670643, 0.656066, 0.136706, 0.164583], [0.639254, 0.618382, 0.199631, 0.240196])) # MEDIUM
print(weighted_iou([0.820532, 0.697243, 0.068833, 0.082230], [0.805022, 0.678125, 0.100000, 0.120956])) # SMALL
print(weighted_iou([0.902880, 0.717892, 0.033973, 0.040931], [0.895362, 0.709050, 0.049453, 0.058860])) # TINY

0.46785733580791994
0.5542859293230449
0.6885386459194439
0.749347676802334


In [7]:
area_covered([0.267645, 0.307458, 0.523305, 0.590944], (1080, 1920))

0.30924394992000004

<h3><span style="color:green;">Test of the <i>iou</i> function.</span></h3>

In [8]:
def test(observed, expected, test_name=""):
    msg = f'({test_name}): Observed: {observed}; Expected: {expected}'
    if (round(observed, 5) == round(expected, 5)): print(f"Test passed ({test_name})")
    else: print(f"Test failed{msg}")

In [9]:
bbox = [0.1, 0.1, 0.2, 0.2]
test(iou(bbox, bbox), 1, "Perfect Overlap") # perfect overlap

bbox2 = [0.9, 0.9, 0.1, 0.1] 
test(iou(bbox, bbox2), 0, "No Overlap") # no overlap

bbox3 = [0.1, 0.2, 0.2, 0.2] 
test(iou(bbox, bbox3), 1/3, "No Overlap") # no overlap

Test passed (Perfect Overlap)
Test passed (No Overlap)
Test passed (No Overlap)


<h2>Computation of the following metrics based on the ground truth (from the test set) and the predictions</h2>
<h3>
    <ul>
        <li>True Positive</li>
        <li>False Positive</li>
        <li><span style="color:grey;">True Negative: not counted (see below)</span></li>
        <li>False Negative</li>
    </ul> 
</h3>

<h3><span style="color:green;">Test of the <i>extract_metrics</i> function.</span></h3>

In [10]:
print("\n------------------------------------")
# perfect match
GT = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
PRED = {"img1": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "perfect match (TP)")
test(fp, 0, "perfect match (FP)")
test(fn, 0, "perfect match (FN)")

print("\n------------------------------------")
# No overlap
GT = {"img2": [[0.8, 0.8, 0.1, 0.1]]}
PRED = {"img2": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap (TP)")
test(fp, 1, "No overlap (FP)")
test(fn, 1, "No overlap (FN)")

print("\n------------------------------------")
# No overlap but bboxes are closer
GT = {"img3": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img3": [[0.2, 0.2, 0.1, 0.1]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 0, "No overlap but bboxes are closer (TP)")
test(fp, 1, "No overlap but bboxes are closer (FP)")
test(fn, 1, "No overlap but bboxes are closer (FN)")

print("\n------------------------------------")
# Not a perfect match but above threshold
GT = {"img4": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img4": [[0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Not a perfect match but above threshold (TP)")
test(fp, 0, "Not a perfect match but above threshold (FP)")
test(fn, 0, "Not a perfect match but above threshold (FN)")

print("\n------------------------------------")
# Two prediction bboxes matching the same ground truth
GT = {"img5": [[0.1, 0.1, 0.1, 0.1]]}
PRED = {"img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# Composed case - 2 pred matching 1 GT - 1 pred matching 1 GT
GT = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]]}
PRED = {"img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 2, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 1, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 0, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# One prediction overlapping two ground truths
GT = {"img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]}
PRED = {"img7": [[0.367304, 0.808140, 0.362103, 0.366419]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "Two prediction bboxes matching the same ground truth (TP)")
test(fp, 0, "Two prediction bboxes matching the same ground truth (FP)")
test(fn, 1, "Two prediction bboxes matching the same ground truth (FN)")

print("\n------------------------------------")
# GT completed included within prediction
GT = {"img8": [[0.5, 0.5, 0.5, 0.5]]}
PRED = {"img8": [[0.5, 0.5, 0.55, 0.55]]}
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 1, "GT completed included within prediction (TP)")
test(fp, 0, "GT completed included within prediction (FP)")
test(fn, 0, "GT completed included within prediction (FN)")


------------------------------------
Test passed (perfect match (TP))
Test passed (perfect match (FP))
Test passed (perfect match (FN))

------------------------------------
Test passed (No overlap (TP))
Test passed (No overlap (FP))
Test passed (No overlap (FN))

------------------------------------
Test passed (No overlap but bboxes are closer (TP))
Test passed (No overlap but bboxes are closer (FP))
Test passed (No overlap but bboxes are closer (FN))

------------------------------------
Test passed (Not a perfect match but above threshold (TP))
Test passed (Not a perfect match but above threshold (FP))
Test passed (Not a perfect match but above threshold (FN))

------------------------------------
Test passed (Two prediction bboxes matching the same ground truth (TP))
Test passed (Two prediction bboxes matching the same ground truth (FP))
Test passed (Two prediction bboxes matching the same ground truth (FN))

------------------------------------
Test passed (Two prediction bboxes

<h3><span style="color:green;">Combined test of the <i>extract_metrics</i> function.</span></h3>

In [11]:
GT = {
    "img1": [[0.2, 0.2, 0.1, 0.1]], 
    "img2": [[0.8, 0.8, 0.1, 0.1]], 
    "img3": [[0.1, 0.1, 0.1, 0.1]],
    "img4": [[0.1, 0.1, 0.1, 0.1]],
    "img5": [[0.1, 0.1, 0.1, 0.1]],
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.8, 0.8, 0.2, 0.3]],
    "img7": [[0.269316, 0.482907, 0.351723, 0.377785], [0.369253, 0.815242, 0.354316, 0.346540]]
    }
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }

tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Combined test (TP)")
test(fp, 4, "Combined test (FP)")
test(fn, 3, "Combined test (FN)")

Test passed (Combined test (TP))
Test passed (Combined test (FP))
Test passed (Combined test (FN))


<h2>Extract the data from the test set (to <i>dict</i>)</h2>

<h3><span style="color:green;">Test of the <i>extract_ground_truth</i> function.</span></h3>

In [12]:
GT = extract_ground_truth(test_set_path="test_set_example")
PRED = {
    "img1": [[0.2, 0.2, 0.1, 0.1]],    # +1 TP
    "img2": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img3": [[0.2, 0.2, 0.1, 0.1]],    # +1 FP, +1 FN
    "img4": [[0.1, 0.1, 0.09, 0.09]],  # +1 TP --> not a perfect match but above threshold
    "img5": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09]], # +1 TP, +1 FP
    "img6": [[0.1, 0.1, 0.1, 0.1], [0.1, 0.1, 0.09, 0.09], [0.8, 0.8, 0.18, 0.32]], # +2 TP, +1 FP
    "img7": [[0.367304, 0.808140, 0.362103, 0.366419]] # +1 TP, +1 FN
    }
tp, fp, fn = extract_metrics(GT, PRED).values()
test(tp, 6, "Extract ground truth from files (TP)")
test(fp, 4, "Extract ground truth from files (FP)")
test(fn, 3, "Extract ground truth from files (FN)")

Test passed (Extract ground truth from files (TP))
Test passed (Extract ground truth from files (FP))
Test passed (Extract ground truth from files (FN))


<h1>Effective code</h1>

<h3>Ground truth extraction.</h3>

In [13]:
GT = extract_ground_truth()

<h3>Predictions extraction</h3>

<h2>Visualisation functions.</h2>

In [ ]:
predictions = predict(model_path, test_set)

<h3>Visualisation support</h3>

In [ ]:
# Example usage:
# predictions = predict(model_path, test_set)
draw_predictions(predictions, GT, test_set)

<h3>Computation of the model performance</h3>

In [ ]:
print(extract_metrics(GT, predictions, t=0.6))

tp, fp, fn = extract_metrics(GT, predictions, t=0.6).values()
print(f"precision={tp/(tp+fp)}")
print(f"recall={tp/(tp+fn)}")

{'true_positives': 718, 'false_positives': 262, 'false_negatives': 475}
precision=0.7326530612244898
recall=0.6018440905280805
